In [1]:
from keras import backend as K
import time
import matplotlib.pyplot as plt
import numpy as np
% matplotlib inline
np.random.seed(2017) 
from keras.models import Sequential
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.layers import Activation, Flatten, Dense, Dropout
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils

Using TensorFlow backend.


In [0]:
from keras.datasets import cifar10
(train_features, train_labels), (test_features, y_labels) = cifar10.load_data()
num_train, img_channels, img_rows, img_cols =  train_features.shape
num_test, _, _, _ =  test_features.shape
num_classes = len(np.unique(train_labels))

In [0]:
train_features = train_features.astype('float32')/255
test_features = test_features.astype('float32')/255
# convert class labels to binary class labels
train_labels = np_utils.to_categorical(train_labels, num_classes)
test_labels = np_utils.to_categorical(y_labels, num_classes)

In [4]:
!wget https://github.com/qubvel/classification_models/archive/master.zip

--2019-07-17 17:28:14--  https://github.com/qubvel/classification_models/archive/master.zip
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/qubvel/classification_models/zip/master [following]
--2019-07-17 17:28:15--  https://codeload.github.com/qubvel/classification_models/zip/master
Resolving codeload.github.com (codeload.github.com)... 192.30.253.120
Connecting to codeload.github.com (codeload.github.com)|192.30.253.120|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘master.zip.4’

master.zip.4            [ <=>                ]  41.40K  --.-KB/s    in 0.03s   

2019-07-17 17:28:15 (1.55 MB/s) - ‘master.zip.4’ saved [42397]



In [9]:
!unzip master.zip

Archive:  master.zip
e27da2b14e390b38dd6fe48f737abc07746bf3af
replace classification_models-master/.gitignore? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: classification_models-master/.gitignore  
  inflating: classification_models-master/.gitmodules  
  inflating: classification_models-master/.travis.yml  
  inflating: classification_models-master/LICENSE  
 extracting: classification_models-master/MANIFEST.in  
  inflating: classification_models-master/README.md  
 extracting: classification_models-master/__init__.py  
  inflating: classification_models-master/classification_models/__init__.py  
  inflating: classification_models-master/classification_models/__version__.py  
 extracting: classification_models-master/classification_models/common/__init__.py  
  inflating: classification_models-master/classification_models/common/blocks.py  
  inflating: classification_models-master/classification_models/keras_applications/__init__.py  
 extracting: classification_models-maste

In [0]:
!cp -r classification_models-master/* ./

In [7]:
!ls

classification_models	      MANIFEST.in   master.zip.3      sample_data
classification_models-master  master.zip    master.zip.4      setup.py
__init__.py		      master.zip.1  README.md	      tests
LICENSE			      master.zip.2  requirements.txt


In [10]:
import keras
from classification_models.resnet import ResNet18, preprocess_input

n_classes = 10

# build model
base_model = ResNet18(input_shape=(32,32,3), weights='imagenet', include_top=False)
x = keras.layers.GlobalAveragePooling2D()(base_model.output)
output = keras.layers.Dense(n_classes, activation='softmax')(x)
model = keras.models.Model(inputs=[base_model.input], outputs=[output])

# train
model.compile(optimizer='SGD', loss='categorical_crossentropy', metrics=['accuracy'])


W0717 17:29:01.520734 140567197185920 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0717 17:29:01.562861 140567197185920 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:245: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0717 17:29:01.564123 140567197185920 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0717 17:29:01.565704 140567197185920 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:181: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.

W0717 17:29:01.567409 140567

In [11]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
data (InputLayer)               (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
bn_data (BatchNormalization)    (None, 32, 32, 3)    9           data[0][0]                       
__________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D (None, 38, 38, 3)    0           bn_data[0][0]                    
__________________________________________________________________________________________________
conv0 (Conv2D)                  (None, 16, 16, 64)   9408        zero_padding2d_1[0][0]           
__________________________________________________________________________________________________
bn0 (Batch

In [12]:
from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(zoom_range=0.0, 
                             horizontal_flip=False)


# Train the model
model_info = model.fit_generator(datagen.flow(train_features, train_labels, batch_size = 128),
                                 samples_per_epoch = train_features.shape[0], nb_epoch = 5, 
                                 validation_data = (test_features, test_labels), verbose=1)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras_pre..., validation_data=(array([[[..., verbose=1, steps_per_epoch=390, epochs=5)`
  # Remove the CWD from sys.path while we load stuff.
W0717 17:29:19.877846 140567197185920 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (f

Epoch 1/5
390/390 [==============================] - 46s 118ms/step - loss: 1.1833 - acc: 0.5954 - val_loss: 0.8839 - val_acc: 0.6921
Epoch 2/5
390/390 [==============================] - 41s 105ms/step - loss: 0.7277 - acc: 0.7463 - val_loss: 0.7505 - val_acc: 0.7426
Epoch 3/5
390/390 [==============================] - 40s 102ms/step - loss: 0.5715 - acc: 0.8011 - val_loss: 0.7226 - val_acc: 0.7561
Epoch 4/5
390/390 [==============================] - 40s 103ms/step - loss: 0.4566 - acc: 0.8424 - val_loss: 0.7088 - val_acc: 0.7625
Epoch 5/5
390/390 [==============================] - 40s 103ms/step - loss: 0.3588 - acc: 0.8777 - val_loss: 0.7169 - val_acc: 0.7683


In [0]:
score = model.evaluate(test_features, test_labels, verbose=0)

In [14]:
print(score)

[0.716928720664978, 0.7683]


In [0]:
y_pred = model.predict(test_features)

In [16]:
print(y_pred)

[[1.6603035e-04 4.5094255e-02 2.2294317e-04 ... 4.6192278e-04
  3.6406177e-03 4.5653647e-03]
 [2.6288056e-03 8.3960213e-02 7.5879466e-06 ... 2.2902201e-04
  9.1183507e-01 1.0988680e-03]
 [6.3662878e-03 8.3551258e-03 1.3228385e-04 ... 6.2301529e-05
  9.8442566e-01 2.1029176e-04]
 ...
 [7.8194253e-06 2.9991932e-07 4.2898289e-04 ... 2.4534680e-03
  1.8344914e-05 4.0850045e-06]
 [1.4152612e-01 8.1472647e-01 1.3723731e-02 ... 1.2695910e-04
  2.3814006e-04 6.3847321e-05]
 [8.2383041e-05 1.2236770e-04 4.0361825e-03 ... 9.0731704e-01
  1.0776268e-04 8.9952839e-05]]


In [0]:
pred =[]
for i in range(len(test_features)):
  pred.append(np.argmax(y_pred[i]))


In [18]:
t_labels = []
for i in range(len(test_features)):
  t_labels.append(np.argmax(test_labels[i]))

pred = np.array(pred)

wrong_pred_id = np.where(pred != t_labels)
wrong_pred = y_labels[pred != t_labels]

print(wrong_pred_id)
print(wrong_pred)



(array([   0,    3,   24, ..., 9983, 9985, 9989]),)
[[3]
 [0]
 [5]
 ...
 [0]
 [5]
 [2]]


In [19]:
print(len(wrong_pred_id))
print(len(wrong_pred))

1
2317


In [0]:
#incorrects = np.nonzero(model.predict(test_features).reshape((-1,)) != y_labels)

In [21]:
np.max(y_pred)

0.9999999